In [1]:
import urllib2 as ul
from bs4 import BeautifulSoup as bs
import time

In [90]:
def HanParse(pageno):
    newspage = ul.urlopen('http://baduk.hangame.com/news.nhn?gseq='+str(pageno)+'&m=view&page=1&searchfield=&leagueseq=0&searchtext=')
    soup = bs(newspage,'html.parser')
    [s.extract() for s in soup(['style', 'script', '[document]', 'head', 'title'])]
    visible_text = soup.getText()
    return visible_text
    

In [128]:
tmp = HanParse(35445)
loc = tmp.find(u'내용')
t2 = [m.start() for m in re.finditer(u'내용', tmp)]
t3 = [m.start() for m in re.finditer(u'관련 뉴스보기', tmp)]
#print tmp[t2[1]:]
print t3
#print tmp[t3[1]:]
print tmp[t2[1]+2:t3[0]]

[4347, 4510]











[전자랜드배] '입담대결'로 피어오른 전설들의 승부
작성자:한창규, 2016-01-23 09:57 입력




■ 인터뷰/ 꿈의 승부 벌이는 바둑전설 5인방

꿈의 승부. 1980~1990년대 세계바둑계를 호령하며 한국바둑을 빛낸 5인의 영웅들이 벌이는 '2016 전자랜드 프라이스킹배 한국바둑의 전설'이 1월 23일부터 2월 14일까지 매주 토요일과 일요일(혹은 월요일까지) 저녁 바둑팬들을 찾아간다. 5인 풀리그로 진행되는 승부는 1인당 4판, 총 10판이 361로 반상을 물들인다. 

그에 앞서 5인의 바둑 전설은 22일 저녁 그랜드힐튼 서울에서 열린 개막식에 한데 모여 화담을 나누면서 정감 넘치는 분위기를 연출했다. 또한 때로는 구수하고, 때로는 위트 넘치는 입담으로 식장의 분위기를 녹였다. 바둑 거장 5인의 인터뷰 내용을 정리했다.
 ▲ 바둑판 뒷면에 새겨진 '전설 5인방'의 휘호.   

■ 조훈현 9단
-대회에 출전하는 소감과 느낌은.
"물론 지금이니까 전설이라 하지만 한참일 때 열렸으면 좋았겠어요. 아까도 누가 우승할 거냐고 하셨는데 당연히 내가 우승할 거라고, 그 당시라면…. 근데 지금은 제일 젊은 사람(이창호)이 우승할 가능성이 많지 않을까. 빨리 좀 만들어 주시지(웃음).
-많은 별명 중에서도 가장 마음에 드는 별명은.
"우리나라에선 '국수'라는 얘기를 많이 듣고 싶어하는데 그게 별명인지 아닌지는 모르겠지만. 그 다음엔 애초에 '제비'였는데 제비는 강남 쪽에서나 통할 거고 '전신'이라는 별명이 제일 듣기 좋았어요. 그것도 뭐, 옛날 이야기가 되어 가지고…. 
-너무 많겠지만 가장 기억에 남는 대국 하나를 꼽는다면.
"가장 기억에 남는 것은 응씨배 우승할 때였고, 진 건 기억 나는 게 너무 많은데, 물론 제자한테 진 것도 가슴 아팠지만 제일 기억에 남는 거라기보다 죄송스러운 것은 중국의 여성기사 루이나이웨이 9단한테 국수전을 빼앗겼거든요, 그것이 남다르고 죄송스럽게 생각합니다."
-160회 타이틀 기록은

In [6]:
newspage = ul.urlopen('http://baduk.hangame.com/news.nhn?gseq=35473&m=view&page=1&searchfield=&leagueseq=0&searchtext=')
soup = bs(newspage,'html.parser')
[s.extract() for s in soup(['style', 'script', '[document]', 'head', 'title'])]
visible_text = soup.getText()

In [68]:
main=ul.urlopen('http://baduk.hangame.com/news.nhn?')
mainbs=bs(main,'html.parser')
mymonths = [link.get('href') for link in mainbs.find_all('a')]


In [129]:
articles=[]
for i in range(1075):
    main=ul.urlopen('http://baduk.hangame.com/news.nhn?&page='+str(i+1))
    mainbs=bs(main,'html.parser')
    tmp = [link.get('href') for link in mainbs.find_all('a')]
    for j in tmp:
        try:
            if j.find('readnews')>0:
                articles.append(j)
        except AttributeError:
            continue

#print articles

mynews=[int(re.search(r'\d+', i).group()) for i in articles]
mynews = sorted(list(set(mynews)))
print len(mynews)

8599


In [132]:
twords=[]
uwords=set()
for i in mynews:
    tmp = HanParse(i)
    #loc = tmp.find(u'내용')
    #t2 = [m.start() for m in re.finditer(u'내용', tmp)]
    #t3 = [m.start() for m in re.finditer(u'관련 뉴스보기', tmp)]
    #if len(t2)>1 and len(t3)>0:
    #    tmp2 = tmp[t2[1]:t3[0]]
    vs = tmp.split()
    for k in vs:
        twords.append(k)
        uwords.add(k)

In [101]:
len(mynews)

800

In [ ]:
tmp = HanParse(35445)
loc = tmp.find(u'내용')
t2 = [m.start() for m in re.finditer(u'내용', tmp)]
print tmp[t2[1]:]

In [71]:
articles=[]
for i in mymonths:
    try:
        if i.find('readnews')>0:
            articles.append(i)
    except AttributeError:
        continue


In [74]:
print articles[3]

javascript:readnews('35477');


In [75]:
import re

In [77]:
[int(re.search(r'\d+', i).group()) for i in articles]

[35480,
 35480,
 35477,
 35477,
 35474,
 35474,
 35473,
 35473,
 35472,
 35472,
 35464,
 35464,
 35462,
 35462,
 35461,
 35461]

In [32]:
vs = visible_text.split()
uwords = set()

In [ ]:
twords=[]
uwords=set()

In [105]:
twords=[]
uwords=set()
for i in mynews:
    tmp = HanParse(i)
    loc = tmp.find(u'내용')
    t2 = [m.start() for m in re.finditer(u'내용', tmp)]
    t3 = [m.start() for m in re.finditer(u'관련 뉴스보기', tmp)]
    tmp2 = tmp[t2[1]:t3[0]]
    vs = tmp2.split()
    for i in vs:
        twords.append(i)
        uwords.add(i)

In [106]:
len(twords),len(uwords)

(415347, 66432)

In [33]:
for i in vs:uwords.add(i)

In [34]:
len(vs),len(uwords)

(1152, 840)

In [109]:
tmp=[]
for i in list(uwords)[:1000]:
    print i,twords.count(i)
    tmp.append((i,twords.count(i)))

리턴매치 21
전국체육대회에서도 1
진중한 1
'준비된 3
끌었다. 7
생산하는 1
유재성4단 1
살아났고, 1
찍었다. 15
참가했지만 1
2승자와 9
한중단체전에서 1
계가한다면 1
메달과 2
오정아-리허는 2
커제가 90
역순으로 2
여자랭킹을 1
강승민ㆍ류수항ㆍ변상일 1
5승2패의 1
3회)이 1
당당한 1
길'을 3
장타오는 3
처리되어 1
앞으로 75
기원합니다." 2
기사들도 1
초석을 2
수업에 3
"거꾸로 1
'톱10' 32
어려 1
넘겼다 1
않았다). 3
지각이었다. 1
패인이었던 1
말도 2
얻었다. 11
랴오위안허(왼쪽)가 1
교류전과 1
보고는) 1
채찍에 1
신안천일염보다 1
국내무대에서 1
우승국에 2
기백을 1
실전적으로 1
견주어 2
설립에 1
속기대국은 1
들른 1
속기대국을 1
(박정환 1
하루새 1
20명이다. 1
덧붙였다. 6
김정현-원성진, 1
●신진서 17
팀이, 1
못하기는 1
-상금이 1
100점을 4
끊어졌고, 1
테스트 3
5분, 3
휘말릴 1
오키드룸 2
위험했다. 2
분석해 1
저장성에서 1
결정될 3
제고는 1
세계바둑오픈] 135
10위권 20
대학교 2
박영훈과 21
육성군은 1
알파고는 6
70수가 1
'제퍼디'에서 1
종국. 1
맞붙었던 4
맞나요?"라고 1
낳은 7
독식했던 1
8-3의 1
분석한 1
'형제대결'로 2
못했으며, 1
젖히는 1
공격하겠다는 1
4위까지의 1
주장이어서 1
신승으로 2
판(기권승)을 1
'바둑소녀' 1
청대에 11
들려주고 1
"미세하긴 1
7집반을 2
바둑'으로 2
중국팬들의 2
7집반의 3
지지옥션의 4
초단)과 1
페어대회 7
농심호'는 1
끼쳤다. 1
부위의 1
트로피가 2
왕쩌진에게 1
160번째이며, 1
137까지 1
딩하오를 2
만지는 1
아성에 3
KB리거입니다" 1
알파고라는 1
'약체'라는 1
덜미 5
공부반'은 1
7집반인 3
맞은편엔 1
챔피언까지 1
황금기(1990~2000년), 2
획득). 2
박정환ㆍ김지석과 4
승점이다. 1
전달했다. 14

In [110]:
tmp=[]
for i in uwords:
    tmp.append((i,twords.count(i)))
zzz=sorted(tmp, key=lambda x: x[1])

In [42]:
zzz=sorted(tmp, key=lambda x: x[1])

In [121]:
aaa = range(10)
aaa.reverse()
print aaa


[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]


In [122]:
zzz.reverse()
for i in zzz[:100]:
    print i[0],i[1]

vs 6624
▲ 5589
관련 4174
9단 2567
- 1748
[전자랜드배 1712
5국 1700
19:00 1601
기보보기 1600
한국바둑의전설] 1600
뉴스보기 1600
있다. 1584
결승 1545
수 1401
있는 1257
첫 1216
2015 1161
두 1091
24강 1058
유창혁 1044
한 978
없습니다. 964
[GS칼텍스배] 896
서봉수 888
만에 884
우승 882
챔피언결정전 875
후 838
[바둑리그] 816
이세돌 809
오늘의 803
내용 803
14:00 801
ㆍ충전20만M*5 800
홍기표(7단) 800
ㆍ바둑자물쇠 800
중계01.31 800
서봉수(9단) 800
유창혁(9단) 800
윤준상(9단) 800
더보기 800
ㆍ베팅횟수 800
입력 800
중국 775
: 761
1위 741
꺾고 730
2국 682
[화제] 681
본선 677
한국 668
작성자:한창규, 660
결과 630
1국 628
이어 616
2차전 610
[단신종합] 592
박정환9단 573
4강 571
이세돌9단 543
그 538
이 537
기보가 536
3국 523
바둑리그] 522
이번 518
KB국민은행 516
크게 514
등 485
팀 474
박정환, 472
이세돌, 472
박정환 471
삼성화재배 446
바둑리그 442
8강 436
커제 433
뉴스가 427
중 424
[명인전] 415
바둑 414
세 411
열린 408
랭킹 406
우승을 406
또 405
연속 402
더 396
간의 396
몽백합배 395
한국기원 391
있으며, 388
함께 388
대회 384
9단이 382
번째 377
바 376
[렛츠런파크배] 375
초단 374
이후 371


In [5]:
print mypage
html = open('21storm.html').read()
soup = BeautifulSoup(html)
[s.extract() for s in soup(['style', 'script', '[document]', 'head', 'title'])]
visible_text = soup.getText()


<html lang="ko">
<head>
<meta charset="utf-8">
<meta content="IE=EmulateIE8" http-equiv="X-UA-Compatible">
<meta content="text/html; charset=utf-8" http-equiv="Content-Type">
<meta content="no-cache" http-equiv="Cache-Control"/>
<meta content="0" http-equiv="Expires"/>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="2JbL9GPK4MRHQzNdDtun9U/Mvo3R2DJEoZidiBWTm9s=" name="verify-v1"/>
<title>한게임 - 즐거운 쉼표, Go 한게임!</title>
<link href="/common/css/renewal_2006/common.css" rel="stylesheet" style="text/css">
<link href="/common/css/renewal_2006/common_layout.css" rel="stylesheet" style="text/css">
<link href="/share/css/group_baduk2009.css" rel="stylesheet" style="text/css">
<link href="/share/css/baduk.css?20140108" rel="stylesheet" type="text/css">
<script src="/common/js/header.js" type="text/javascript"></script>
<script charset="UTF-8" src="/common/clientscript/hsf.js" type="text/javascript"></script>
<script charset="UTF-8" src="/common/clientscript/conv/common.js" type="tex

In [134]:
len(twords),len(uwords)

(5575082, 357660)

In [135]:
len(zzz)

66432

In [146]:
f=open('hanwords.u','w')
for i in uwords:
    f.write(i.encode('utf8')+'\n')
g=open('hanwords.t','w')
for i in twords:
    g.write(i.encode('utf8')+'\n')

In [140]:
for i in list(uwords)[:100]:print i

"사활
여류아마연승최강전의
전국체육대회에서도
1위(박정환),
대마에서부터
2승자와
계가한다면
9907점으로
제일화재]
진격하지
"강진군
대결(기사
자극을
길'을
사줘야
연구판을
피곤했어요."
자연경관에
배열한다.
말씀하셨는데
9단(5위)
뭐하지만
눈빛이며
주역'을
이세돌-황이중,
얻었다.
바뀌어선
기사(조혜연,
발산시키는
가렸다
채찍에
제일화재,
우승국엔
제일화재.
제일화재)
주니어부에
우승국에
자충에
제일화재!
기술고시를
개화한
ckha3145(6급)은
호성적과
55명).
하루새
토로한다고
당하므로
승부사'로
시비거리를
22승12패의
윤준상(4승)
위험했다.
자극이
위험했다!
고려가
등극,
등극.
4지명자인
5단(유럽),
등극!
열여덟까지
테이블
기보해설,
6연승자로
1위(6승2패)로
약호입니다.
홍승희
후배기사들과
'인상파
출전중인
네덜란드를
ŔúżěÁĺ˝ŽżĄ°Ő
백102ㆍ104로
차이나'라지.
건강에
연출자는
대표신예
KT(회장
160번째이며,
306살이다.
<제1국>신민준-이태현.
김학도-프로기사
`개척자'
안형존에
영애의
접어들었고,
한쪽도
들어갔다"면서
품어내는
종목에선
만년패에
1호점
벌려가자
일침
시상대의
스물두
원장(청담어린이집)을
내기바둑을
3번(7ㆍ8ㆍ14),
일반입단대회가


In [145]:
for i in words[:100]:print i

"사활

여류아마연승최강전의

전국체육대회에서도

1위(박정환),

대마에서부터

2승자와

계가한다면

9907점으로

제일화재]

진격하지

"강진군

대결(기사

자극을

길'을

사줘야

연구판을

피곤했어요."

자연경관에

배열한다.

말씀하셨는데

9단(5위)

뭐하지만

눈빛이며

주역'을

이세돌-황이중,

얻었다.

바뀌어선

기사(조혜연,

발산시키는

가렸다

채찍에

제일화재,

우승국엔

제일화재.

제일화재)

주니어부에

우승국에

자충에

제일화재!

기술고시를

개화한

ckha3145(6급)은

호성적과

55명).

하루새

토로한다고

당하므로

승부사'로

시비거리를

22승12패의

윤준상(4승)

위험했다.

자극이

위험했다!

고려가

등극,

등극.

4지명자인

5단(유럽),

등극!

열여덟까지

테이블

기보해설,

6연승자로

1위(6승2패)로

약호입니다.

홍승희

후배기사들과

'인상파

출전중인

네덜란드를

ŔúżěÁĺ˝ŽżĄ°Ő

백102ㆍ104로

차이나'라지.

건강에

연출자는

대표신예

KT(회장

160번째이며,

306살이다.

<제1국>신민준-이태현.

김학도-프로기사

`개척자'

안형존에

영애의

접어들었고,

한쪽도

들어갔다"면서

품어내는

종목에선

만년패에

1호점

벌려가자

일침

시상대의

스물두

원장(청담어린이집)을

내기바둑을

3번(7ㆍ8ㆍ14),

일반입단대회가



In [147]:
a=[1,1,1,2,3,4,5]

In [150]:
a.remove(1)

AttributeError: 'list' object has no attribute 'removeall'

In [149]:
def remove_values_from_list(the_list, val):
    for ii in range(the_list.count(val)):
        the_list.remove(val)

[1, 1, 2, 3, 4, 5]

In [157]:
safewords=list(twords)

In [158]:
tmp=[]
for i in uwords:
    n=safewords.count(i)
    for j in range(n):
        safewords.remove(i)
    tmp.append((i,n))

KeyboardInterrupt: 

In [160]:
zzz=sorted(tmp, key=lambda x: x[1])

In [156]:
len(uwords)

357660

In [165]:
import codecs

In [169]:
zzz.reverse()
h=codecs.open('hanwords.s','w',encoding='utf-8')
for a in zzz:
    h.write(unicode(a[0])+' '+str(a[1])+'\n')

In [159]:
len(tmp)

51418

In [163]:
tmp[0][0]

u'"\uc0ac\ud65c'

In [167]:
zzz

[(u'Plus', 60193),
 (u'\u25b2', 37617),
 (u'-', 23401),
 (u'\uc788\ub294', 21643),
 (u'\ud55c\uad6d\ubc14\ub451\uc758\uc804\uc124]', 17178),
 (u'\uc774\uc138\ub3cc', 10834),
 (u'[GS\uce7c\ud14d\uc2a4\ubc30]', 10141),
 (u'9\ub2e8\uc774', 9491),
 (u'\uc788\uc9c0', 8775),
 (u'\uc990\uae38', 8681),
 (u'\uc54a\uc2b5\ub2c8\ub2e4.', 8671),
 (u'\uc990\uac70\uc6c0\uc744', 8661),
 (u'14:00', 8633),
 (u'[Plus]', 8599),
 (u'[Plus', 8599),
 (u'\uc785\uc7a5\ud558\uae30', 8593),
 (u'\uacf5\uc9c0\uc0ac\ud56d', 8590),
 (u'\uc804\uc6a9\uc0c1\uc810', 8589),
 (u'\uc548\ub0b4/\uc2e0\uccad', 8589),
 (u'\ubc18\uc0c1\uc5ec\ud589,\ud55c\uac8c\uc784', 8589),
 (u'\uc0c1\uc810', 8589),
 (u'\ud68c\uc6d0\uc81c\uc5d0', 8589),
 (u'\ud50c\ub7ec\uc2a4\ud68c\uc6d0\ub9cc\uc744', 8589),
 (u'\uba38\ub2c8,', 8589),
 (u'\uc18c\uc561\uacb0\uc81c', 8589),
 (u'\u318d\ubca0\ud305\ud69f\uc218', 8589),
 (u'\uc2b9\uc790\ub9de\ud788\uae30', 8589),
 (u'[\uacf5\uc9c0\uc0ac\ud56d]', 8589),
 (u'\uc774\ucc3d\ud638', 6873),
 (u'\uae30\ubc